In [ ]:
import numpy as np
import elboflow as ef
import tensorflow as tf
import scipy.stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Generate some data for linear regression
np.random.seed(1)
num_samples = 100
num_dims = 2

feature_means = np.random.normal(0, 1, num_dims)
x = ef.add_bias(np.random.normal(0, 1, (num_samples, num_dims)) + feature_means)
theta = np.random.normal(0, 1, num_dims + 1)
predictor = np.dot(x, theta)
tau = np.random.gamma(1)
y = predictor + np.random.normal(0, 1 / np.sqrt(tau), num_samples)

In [ ]:
# Construct a graph
with tf.Graph().as_default() as graph:
    # Define the factors
    q_theta = ef.MultiNormalDistribution(
        ef.get_variable('theta_mean', num_dims + 1),
        ef.get_positive_definite_variable('theta_log_precision', (num_dims + 1, num_dims + 1))
    )
    q_tau = ef.GammaDistribution(
        ef.get_positive_variable('tau_shape', []),
        ef.get_positive_variable('tau_scale', []),
    )
    
    # Evaluate the expected log joint distribution
    log_likelihood = tf.reduce_sum(ef.NormalDistribution.linear_log_likelihood(y, x, q_theta, q_tau))
    log_joint = log_likelihood + \
        ef.MultiNormalDistribution.log_likelihood(q_theta, np.zeros(num_dims + 1), 1e-3 * np.eye(num_dims + 1)) + \
        ef.GammaDistribution.log_likelihood(q_tau, 1e-3, 1e-3)
    # Add the entropy
    elbo = log_joint + q_theta.entropy + q_tau.entropy
    
    # Add a training operation
    train_op = tf.train.AdamOptimizer(0.1).minimize(-elbo)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
sess.run(elbo)

In [ ]:
# Maximize the ELBO
elbos = []

for _ in tqdm_notebook(range(2000)):
    _, _elbo = sess.run([train_op, elbo])
    elbos.append(_elbo)
    
plt.plot(-np.asarray(elbos))
plt.yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

with graph.as_default():
    ef.plot_pdf(sess, q_theta, reference=theta, ax=ax1)
    ef.plot_pdf(sess, q_tau, reference=tau, ax=ax2)

In [ ]:
with graph.as_default():
    foo = ef.plot_comparison(sess, q_theta, theta, ax=plt.gca())

In [ ]:
with graph.as_default():
    ef.plot_cov(sess, q_theta)
    print(np.mean(x, axis=0))